# Ultralytics Classifier

Set up runtime to use GPU

## Create or get the dataset

In [1]:
# https://www.mvtec.com/company/research/datasets/mvtec-ad/downloads
# Get bottle dataset
!wget https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937370-1629951468/bottle.tar.xz

--2024-10-16 16:31:54--  https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937370-1629951468/bottle.tar.xz
Resolving www.mydrive.ch (www.mydrive.ch)... 91.214.169.64
Connecting to www.mydrive.ch (www.mydrive.ch)|91.214.169.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155880244 (149M) [application/x-xz]
Saving to: ‘bottle.tar.xz’

bottle.tar.xz       100%[===================>] 148.66M  19.3MB/s    in 9.0s    

2024-10-16 16:32:04 (16.4 MB/s) - ‘bottle.tar.xz’ saved [155880244/155880244]



In [2]:
!tar -xf bottle.tar.xz

In [ ]:
!pip install ultralytics

In [51]:
import os
import shutil
import random
import yaml
# pathlib

def get_filenames(folder):
    filenames = []
    for f in folder:
        for file in os.listdir(f):
            filename = os.path.join(f, file)
            filenames.append(filename)
    return filenames

def split_train_test_val(filenames, train_pct, test_pct, val_pct):
    random.shuffle(filenames)

    # Calculate split indices
    total_files = len(filenames)
    split_train = int(train_pct * total_files)
    split_val = split_train + int(val_pct * total_files)

    # Split filenames
    train_filenames = filenames[:split_train]
    val_filenames = filenames[split_train:split_val]
    test_filenames = filenames[split_val:]
    return train_filenames, val_filenames, test_filenames

shutil.rmtree("dataset")

folder_ok = ["bottle/train/good"]
#folder_nok = ["bottle/test/broken_large","bottle/test/broken_small","bottle/test/contamination"]
folder_broken_large = ["bottle/test/broken_large"]
folder_broken_small = ["bottle/test/broken_small"]
folder_contamination = ["bottle/test/contamination"]

filenames_nok = get_filenames(folder_nok)
filenames_ok = get_filenames(folder_ok)
filenames_broken_large = get_filenames(folder_broken_large)
filenames_broken_small = get_filenames(folder_broken_small)
filenames_contamination = get_filenames(folder_contamination)

d = {
    "ok": filenames_ok,
    "broken_large": filenames_broken_large,
    "broken_small": filenames_broken_small,
    "contamination": filenames_contamination
}
classes = list(d.keys())
for clas in classes:
    train_filenames, val_filenames, test_filenames = split_train_test_val(d[clas], 0.7,0.2,0.1)
    tasks = ["train", "test","val"]
    for task in tasks:
        folder_dst= f"dataset/{task}/{clas}"
        os.makedirs(folder_dst, exist_ok=True)
        if task == "train":
            for f in train_filenames:
                shutil.copy(f, folder_dst)
        if task == "test":
            for f in test_filenames:
                shutil.copy(f, folder_dst)
        if task == "val":
            for f in val_filenames:
                shutil.copy(f, folder_dst)

dataset = "/content/dataset"
data = {
    "train": f"{dataset}/train",
    "val":   f"{dataset}/val",
    "test":  f"{dataset}/test",
    "nc": len(classes),
    "names": classes,
}
# Create and write to the YAML file
yaml_file_path = f"{dataset}/data.yaml"
with open(yaml_file_path, 'w') as file:
    yaml.dump(data, file, sort_keys=False)


In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8m-cls.pt")
results = model.train(data="/content/dataset", epochs=30, imgsz=224,batch=4)

## Inference


In [ ]:
import torch
import numpy as np

class Yolov8Inference:
    def __init__(self, model_path, gpu=False):
        self.model = YOLO(model_path)
        self.device = torch.device("cpu")
        if gpu:
            if torch.cuda.is_available():
                self.device = torch.device("cuda")
                print(f"Model will run on GPU: {torch.cuda.get_device_name(0)}")
            else:
                print("Model will run on CPU, because GPU is not available")
        else:
            print("Model will run on CPU")
        self.labels = self.model.names

    def predict(self, im):

        # im has to be numpy array
        # start = time.time()
        res = self.model.predict(im, imgsz=640, device=self.device, conf=0.01, iou=0.01, verbose=False)
        # end = time.time()
        # print(f"Model inference: {end-start:2f}s")

        #print(res[0].probs)
        names = res[0].names
        scores = res[0].probs.data.cpu().numpy()

        im_draw = im.copy()
        idx = np.argmax(scores)
        score = scores[idx]
        cv2.putText(im_draw, f"{names[idx]}: {score:.2f}", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.5,(0, 0, 255), 1)

        return im_draw



In [ ]:
model = Yolov8Inference("/content/tmp_training/train/weights/best.pt")
for f in filename_ok:
    im = cv2.imread(f, cv2.IMREAD_COLOR)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im_draw = model.predict(im)
    display(Image.fromarray(im_draw).resize((512,512)))
    break